<a href="https://colab.research.google.com/github/SahilDhull/emphasis_selection/blob/master/model/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
!pip install config

In [0]:
import codecs
import numpy as np

In [106]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

file_path = 'drive/My Drive/datasets/ensemble/bert_large/'
save_path = file_path

vfile1 = file_path + 'val1.txt'
vfile2 = file_path + 'val2.txt'

tfile1 = file_path + 'test1.txt'
tfile2 = file_path + 'test2.txt'

Mounted at /content/drive


In [0]:
def read_map(file, map):
  
  with codecs.open(file, 'r', 'utf-8') as f:
      lines = f.readlines()

  flag = True

  d = [[]]

  i = 0
  
  for line in lines:
    if not (line.isspace()):
      feats = line.strip().split()
      d.append(feats)
      flag = False
    
    elif flag == False:
      d.pop(0)
      map[i] = d
      # print(d)
      i = i + 1
      flag = True
      d = [[]]
    
  print(i)


In [101]:
dict1 = {}
dict2 = {}

read_map(vfile1,dict1)
read_map(vfile2,dict2)

td1 = {}
td2 = {}

read_map(tfile1,td1)
read_map(tfile2,td2)

# print(dict1)
# print(dict2)

val_length = 392
test_length = 743

392
392
743
743


In [0]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def match_M(batch_scores_no_padd, batch_labels_no_pad):

    top_m = [1, 2, 3, 4]
    batch_num_m=[]
    batch_score_m=[]
    for m in top_m:
        intersects_lst = []
        # exact_lst = []
        score_lst = []
        ############################################### computing scores:
        for s in batch_scores_no_padd:
            if len(s) <=m:
                continue
            h = m
            # if len(s) > h:
            #     while (s[np.argsort(s)[-h]] == s[np.argsort(s)[-(h + 1)]] and h < (len(s) - 1)):
            #         h += 1

            # s = np.asarray(s.cpu())
            s = np.asarray(s)
            #ind_score = np.argsort(s)[-h:]
            ind_score = sorted(range(len(s)), key = lambda sub: s[sub])[-h:]
            score_lst.append(ind_score)

        ############################################### computing labels:
        label_lst = []
        for l in batch_labels_no_pad:
            if len(l) <=m:
                continue
            # if it contains several top values with the same amount
            h = m
            # l = l.cpu()
            if len(l) > h:
                while (l[np.argsort(l)[-h]] == l[np.argsort(l)[-(h + 1)]] and h < (len(l) - 1)):
                    h += 1
            l = np.asarray(l)
            ind_label = np.argsort(l)[-h:]
            label_lst.append(ind_label)

        ############################################### :

        for i in range(len(score_lst)):
            intersect = intersection(score_lst[i], label_lst[i])
            intersects_lst.append((len(intersect))/(min(m, len(score_lst[i]))))
            # sorted_score_lst = sorted(score_lst[i])
            # sorted_label_lst =  sorted(label_lst[i])
            # if sorted_score_lst==sorted_label_lst:
            #     exact_lst.append(1)
            # else:
            #     exact_lst.append(0)
        batch_num_m.append(len(score_lst))
        batch_score_m.append(sum(intersects_lst))
    return batch_num_m, batch_score_m

def print_to_file(file_path, var):
  with open(save_path + file_path, "w") as text_file:
    text_file.write(var)

In [95]:
num_m = [0, 0, 0, 0]
score_m = [0, 0, 0, 0]

cnt = 0

all_labels = []
all_our_labels = []

for i in range(val_length):
  d1 = dict1[i]
  d2 = dict2[i]

  labels = []
  our_labels = []

  for j in range(len(d1)):
    # print(d1[j])
    label = (float)(d1[j][3])
    labels.append(label)
    our_label1 = (float)(d1[j][2])
    our_label2 = (float)(d2[j][2])
    ls = [our_label1, our_label2]
    our_label = np.mean(ls)
    our_labels.append(our_label)
    # print(type(d1[j][2]))
    # break

  # print(labels)
  # print(our_labels)

  cnt = cnt + 1
  all_labels.append(labels)
  all_our_labels.append(our_labels)

  if cnt%1==0:
    # nlabels = np.array(labels).reshape(1,len(labels))
    # nour_labels = np.array(our_labels).reshape(1,len(our_labels))

    # print(all_labels)

    batch_num_m, batch_score_m = match_M(all_our_labels, all_labels)

    num_m = [sum(i) for i in zip(num_m, batch_num_m)]
    # print(num_m)
    score_m = [sum(i) for i in zip(score_m, batch_score_m)]
    # print(score_m)

    all_labels = []
    all_our_labels = []

  # print (d1)
  # print (d2)
  # break

m_score = [i/j for i,j in zip(score_m, num_m)]

print(m_score)
v_score = np.mean(m_score)
print(v_score)

[0.673469387755102, 0.7917771883289124, 0.8361742424242421, 0.8598484848484849]
0.7903173258391853


In [0]:
s = ""
sent_id = ""
k = 0

for i in range(test_length):
  d1 = td1[i]
  d2 = td2[i]

  our_labels = []

  s = s + "\n"

  for j in range(len(d1)):
    our_label1 = (float)(d1[j][2])
    our_label2 = (float)(d2[j][2])
    ls = [our_label1, our_label2]
    our_label = np.mean(ls)

    s = s + "{}\t{}\t{}\t".format(d1[j][0],d1[j][1],our_label) + "\n"
  
  # s = s + "\n"
    



In [104]:
print(s)


S_0_0	We	0.06885289959609509	
S_0_1	'll	0.04543304070830345	
S_0_2	be	0.07225720584392548	
S_0_3	closed	0.446710541844368	
S_0_4	from	0.04671648144721985	
S_0_5	12/24	0.26124394685029984	
S_0_6	to	0.09719544276595116	
S_0_7	1/1	0.3445126414299011	
S_0_8	.	0.048845697194337845	
S_0_9	See	0.1955564245581627	
S_0_10	you	0.15675046667456627	
S_0_11	in	0.07856473326683044	
S_0_12	the	0.0830814503133297	
S_0_13	New	0.4375517964363098	
S_0_14	Year	0.49775081872940063	
S_0_15	!	0.15357784181833267	

Q_1_0	No	0.10015739873051643	
Q_1_1	matter	0.13169266656041145	
Q_1_2	how	0.1839146539568901	
Q_1_3	hard	0.5648887753486633	
Q_1_4	you	0.2619151622056961	
Q_1_5	work	0.6142759025096893	
Q_1_6	,	0.058471424505114555	
Q_1_7	someone	0.31316132843494415	
Q_1_8	else	0.39144250750541687	
Q_1_9	is	0.21737606078386307	
Q_1_10	working	0.6015322804450989	
Q_1_11	harder	0.7231703996658325	
Q_1_12	.	0.2534143328666687	

Q_2_0	The	0.04486728832125664	
Q_2_1	less	0.6354193985462189	
Q_2_2	I	0.18576686084270477	

In [0]:
print_to_file('en_test.txt',s)